# Software-Defined Networks Emulation in Mininet

Mininet is a network emulator that allows for rapid development of vast network architectures. Its limits lie with the hardware limits of the platform that runs the mininet. In this course, we will discuss some basic usage of mininet to emulate various network topologies.

**IMPORTANT:** Mininet's *mn* wrapper works as a CLI so we cannot use it in the Jupyter. Instead, please use the remote console to the servers having the mininet package installed and ready. These servers are at:

You can log in using standard ssh connection. From the terminal in your computer please use the following command.

```bash
ssh student@10.100.0.XX
ssh student@158.196.244.134 -p XX22  

XX: 41-53
```
The password for the connection is **student**.

To kill a process using its port, you can follow these general steps:

sudo lsof -i :6653 (find process with 6653 port)

sudo kill -9 PID  (PID = PID of your process)






## Running Mininet

Mininet can be run using its Python API. However, for most cases we use an interactive console provided by the *mn* wrapper. The simplest topology mininet can run can be invoked by the following command:

```bash
mn
```

This command invokes the mininet wrapper that runs a Python virtual environment and creates the topology with single switch and two hosts that are connected to it. This is depicted in the figure below.

![topo](fig/simple.png)

The switch is an instance of Open vSwitch. A controller is and internal reference implementation of the Stanford controller and the hosts are run as separate processes so that they can be used to run even some more complex applications.

Mininet outputs some useful information:

```text
*** Creating network
*** Adding controller
*** Adding hosts:
h1 h2 
*** Adding switches:
s1 
*** Adding links:
(h1, s1) (h2, s1) 
*** Configuring hosts
h1 h2 
*** Starting controller
c0 
*** Starting 1 switches
s1 ...
*** Starting CLI:
```

It creates a network, adds a controller, hosts h1 and h2 and then the switch. The links in the topology are between hosts and the switch. Please note the names of the elements, such as h1, s1, or c0. These can be used to identify the elements in the network.

To **exit** from the mininet console, just type **exit**.

In case of mininet crash, just type **mn -c** for cleanup.

Let's now explore some basic functionalities.

## Hosts act as real Linux servers

Hosts can be treated similarly as usual Linux servers. Let's see the IP configuration of the host h1.

```bash
mininet> h1 ip a
```

This will output very familiar information:

```text
1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: h1-eth0@if11: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default qlen 1000
    link/ether ea:21:6a:29:c0:5e brd ff:ff:ff:ff:ff:ff link-netnsid 0
    inet 10.0.0.1/8 brd 10.255.255.255 scope global h1-eth0
       valid_lft forever preferred_lft forever
    inet6 fe80::e821:6aff:fe29:c05e/64 scope link 
       valid_lft forever preferred_lft forever
```

Here, you can see that the host actually has a network interface with both MAC and IP addresses assigned. This is and example of how to run a single command on the emulated host. But, what if we need to do some more complex stuff? We can switch to the host's console by invoking **bash**.

```bash
mininet> h1 bash
```

There, you can for example run a tcpdump command. If you want to return to the mininet console, just type **exit**.

## Get or set the switch params

To get some information about the switch being used, we can use a **dpctl** command. It is actually an internal alias of the mininet for the **ovs-ofctl** command. Both can be used to obtain information about the switches and/or to work with the flow/group/meter table entries.

So, the most basic information is the information about the version of the switch and its type. This can be obtained by:

```bash
mininet> dpctl dump-desc
```

This will output the following (or similar).

```text
*** s1 ------------------------------------------------------------------------
OFPST_DESC reply (xid=0x2):
Manufacturer: Nicira, Inc.
Hardware: Open vSwitch
Software: 2.10.1
Serial Num: None
DP Description: None
```

The equivalent using the ovs-ofctl command would look as follows.

```bash
ovs-ofctl dump-desc s1
```

The information about the switch physical and registered ports can be reached by invoking:

```bash
mininet> dpctl show
```

The output then can look like following:

```text
*** s1 ------------------------------------------------------------------------
OFPT_FEATURES_REPLY (xid=0x2): dpid:0000000000000001
n_tables:254, n_buffers:0
capabilities: FLOW_STATS TABLE_STATS PORT_STATS QUEUE_STATS ARP_MATCH_IP
actions: output enqueue set_vlan_vid set_vlan_pcp strip_vlan mod_dl_src mod_dl_dst mod_nw_src mod_nw_dst mod_nw_tos mod_tp_src mod_tp_dst
 1(s1-eth1): addr:26:98:6e:44:a0:70
     config:     0
     state:      0
     current:    10GB-FD COPPER
     speed: 10000 Mbps now, 0 Mbps max
 2(s1-eth2): addr:2a:7d:0b:34:61:95
     config:     0
     state:      0
     current:    10GB-FD COPPER
     speed: 10000 Mbps now, 0 Mbps max
 LOCAL(s1): addr:b2:26:ba:70:07:40
     config:     PORT_DOWN
     state:      LINK_DOWN
     speed: 0 Mbps now, 0 Mbps max
OFPT_GET_CONFIG_REPLY (xid=0x4): frags=normal miss_send_len=0
```

This is actually the information that the switch sends using the Openflow protocol. You can observe it using the **tcpdump** running on the remote computer.

You can ask for port stats as well.

```bash
mininet> dpctl dump-ports 
```

```text
*** s1 ------------------------------------------------------------------------
OFPST_PORT reply (xid=0x2): 3 ports
  port LOCAL: rx pkts=0, bytes=0, drop=23, errs=0, frame=0, over=0, crc=0
           tx pkts=0, bytes=0, drop=0, errs=0, coll=0
  port  "s1-eth1": rx pkts=13, bytes=1006, drop=0, errs=0, frame=0, over=0, crc=0
           tx pkts=24, bytes=1852, drop=0, errs=0, coll=0
  port  "s1-eth2": rx pkts=12, bytes=936, drop=0, errs=0, frame=0, over=0, crc=0
           tx pkts=25, bytes=1922, drop=0, errs=0, coll=0
```

You can even explore the flow tables.

```bash
mininet> dpctl dump-tables
```

```text
*** s1 ------------------------------------------------------------------------
OFPST_TABLE reply (xid=0x2):
  table 0:
    active=1, lookup=24, matched=24
    max_entries=1000000
    matching:
      in_port: exact match or wildcard
      eth_src: exact match or wildcard
      eth_dst: exact match or wildcard
      eth_type: exact match or wildcard
      vlan_vid: exact match or wildcard
      vlan_pcp: exact match or wildcard
      ip_src: exact match or wildcard
      ip_dst: exact match or wildcard
      nw_proto: exact match or wildcard
      nw_tos: exact match or wildcard
      tcp_src: exact match or wildcard
      tcp_dst: exact match or wildcard

  table 1:
    active=0, lookup=0, matched=0
    (same features)

  table 2: ditto
  table 3: ditto
  ...
```

Please, see the information about the number of lookups and matches. This is related to the packets that traversed the switch.

You can test the connection to the controller by OpenFlow ping as well.

```bash
mininet> dpctl ping
```

```text
*** s1 ------------------------------------------------------------------------
64 bytes from s1: xid=00000002 time=0.2 ms
64 bytes from s1: xid=00000003 time=0.1 ms
64 bytes from s1: xid=00000004 time=0.1 ms
64 bytes from s1: xid=00000005 time=0.1 ms
64 bytes from s1: xid=00000006 time=0.1 ms
64 bytes from s1: xid=00000007 time=0.1 ms
64 bytes from s1: xid=00000008 time=0.1 ms
64 bytes from s1: xid=00000009 time=0.1 ms
64 bytes from s1: xid=0000000a time=0.1 ms
64 bytes from s1: xid=0000000b time=0.1 ms
```

To dig information about the current entries in the flow tables, you can use the following command. The output of this command can differ in case of you making some traffic between the hosts.

```bash
mininet> dpctl dump-flows  
```

```text
*** s1 ------------------------------------------------------------------------
 cookie=0x0, duration=2.572s, table=0, n_packets=10, n_bytes=852, priority=0 actions=CONTROLLER:128
```

If you try to run a ping between the host such as

```bash
mininet> h1 ping h2
```

You will see that there are two more flow entries in the flow tables.

```text
*** s1 ------------------------------------------------------------------------
 cookie=0x0, duration=2.422s, table=0, n_packets=1, n_bytes=98, idle_timeout=60, priority=1,icmp,in_port="s1-eth1",vlan_tci=0x0000/0x1fff,dl_src=5a:60:5b:e1:3d:38,dl_dst=1e:67:69:87:e2:39,nw_src=10.0.0.1,nw_dst=10.0.0.2,nw_tos=0,icmp_type=8,icmp_code=0 actions=output:"s1-eth2"
 cookie=0x0, duration=2.421s, table=0, n_packets=1, n_bytes=98, idle_timeout=60, priority=1,icmp,in_port="s1-eth2",vlan_tci=0x0000/0x1fff,dl_src=1e:67:69:87:e2:39,dl_dst=5a:60:5b:e1:3d:38,nw_src=10.0.0.2,nw_dst=10.0.0.1,nw_tos=0,icmp_type=0,icmp_code=0 actions=output:"s1-eth1"
 cookie=0x0, duration=1105.177s, table=0, n_packets=33, n_bytes=2462, priority=0 actions=CONTROLLER:128
```

Please note the match fields and the timeouts.

You can delete all the flow entries simply by issuing:

```bash
mininet> dpctl del-flows
```

Please, now try to ping the host h2 from host h1. This will not work.

### Question

Why the ping is not working anymore?

Let's try to add a simple rule that forwards all the traffic coming from port 1 to port 2 and vice versa.

```bash
mininet> dpctl add-flow in_port=2,actions:output=1
mininet> dpctl add-flow in_port=1,actions:output=2
```

Now, try to run the ping again. What flow entries can you see?

## Mininet commands

Mininet has some handy commands to get the info about the nodes, the links between nodes and process numbers running the instances of the hosts, controller or switches. 

### Task

Please try to issue the commands below and investigate the output.

```bash
mininet> nodes
mininet> net
mininet> dump
```

## Application and link performance testing

You can also run the performance test of the line between the nodes with iperf like this:

```bash
mininet> h2 iperf -c h1
```

The first line runs an iperf server, the second runs a client.

### Task
Investigate the output of the iperf command.

## Custom topologies and link parameters

There are several ways how to automatically create the most common topologies in mininet (please see man mn). However, in cases when you need a special topology and/or emulate delays and/or losses on the links between the nodes the custom topology definition becomes a must.

To define a custom topology, you can use a sceleton class in the cell below.

In [ ]:
"""Custom topology example
Adding the 'topos' dict with a key/value pair to generate our newly defined
topology enables one to pass in '--topo=mytopo' from the command line.
"""

from mininet.topo import Topo

class MyTopo( Topo ):
    "Simple topology example."

    def __init__( self ):
        "Create custom topo."

        # Initialize topology
        Topo.__init__( self )

        # Add hosts and switches
        # your code here

        # Add links
        # your code here

topos = { 'mytopo': ( lambda: MyTopo() ) }


The topology we are about to model is depicted in the figure below.

![custom topo](fig/custom.png)

First, we need to add hosts.

In [ ]:
h1 = self.addHost('h1', ip='10.1.0.11/8')
h2 = self.addHost('h2', ip='10.1.0.22/8')
h3 = self.addHost('h3', ip='10.1.0.33/8')

Then, let's add switches.

In [ ]:
s1 = self.addSwitch('s1')
s2 = self.addSwitch('s2')
s3 = self.addSwitch('s3')

And then, we need to wire the interfaces together.

In [ ]:
self.addLink(h1, s1)
self.addLink(h2, s2)
self.addLink(h3, s3)

self.addLink(s1, s2, bw=10, delay="30ms")
self.addLink(s1, s3, bw=15, delay="50ms")

The links are actually a tuples of originating and destination nodes. We can add traffic control params to links as well, where **bw** stands for bandwidth in Mbit/s and delay stands for network link delay. You can use **loss** as well. In this case, the value is in percent and it is an equally distributed loss pattern.

Complete topology definition would then look as follows.

In [ ]:
"""Custom topology example
Adding the 'topos' dict with a key/value pair to generate our newly defined
topology enables one to pass in '--topo=mytopo' from the command line.
"""

from mininet.topo import Topo

class MyTopo( Topo ):
    "Simple topology example."

    def __init__( self ):
        "Create custom topo."

        # Initialize topology
        Topo.__init__( self )

        # Add hosts and switches
        h1 = self.addHost('h1', ip='10.1.0.11/8')
        h2 = self.addHost('h2', ip='10.1.0.22/8')
        h3 = self.addHost('h3', ip='10.1.0.33/8')
        
        s1 = self.addSwitch('s1')
        s2 = self.addSwitch('s2')
        s3 = self.addSwitch('s3')

        # Add links
        self.addLink(h1, s1)
        self.addLink(h2, s2)
        self.addLink(h3, s3)

        self.addLink(s1, s2, bw=10, delay="30ms")
        self.addLink(s1, s3, bw=15, delay="50ms")

topos = { 'mytopo': ( lambda: MyTopo() ) }

To run this topology, you can use the following command, which assumes that the file with the topology is called **ex.py**. **Important** for the network parameters to work as expected, **the link type must be set to tc**.

```bash
mn --custom=ex.py --topo mytopo --mac --link tc
```

### Task

* Add network loss to the topology as well.
* Measure the network performance using iperf tool.

In [ ]:
##Topology with losess
from mininet.topo import Topo

class MyTopo(Topo):
    "Simple topology example."

    def __init__(self):
        "Create custom topo."

        # Initialize topology
        Topo.__init__(self)

        # Add hosts and switches
        h1 = self.addHost('h1', ip='10.1.0.11/8')
        h2 = self.addHost('h2', ip='10.1.0.22/8')
        h3 = self.addHost('h3', ip='10.1.0.33/8')

        s1 = self.addSwitch('s1')
        s2 = self.addSwitch('s2')
        s3 = self.addSwitch('s3')

        # Add links
        self.addLink(h1, s1)
        self.addLink(h2, s2)
        self.addLink(h3, s3)

        # Add links with bandwidth, delay, and loss
        self.addLink(s1, s2, bw=10, delay="30ms", loss=2)  # 2% packet loss
        self.addLink(s1, s3, bw=15, delay="50ms", loss=5)  # 5% packet loss

topos = {'mytopo': (lambda: MyTopo())}
